In [167]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client()

/Users/robords/.venv/big_query_ml/lib/python3.7/site-packages/google/auth/_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/robords/.venv/big_query_ml/lib/python3.7/site-packages/google/auth/_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more inform

### Get the list of tables in the dataset

In [3]:
dataset_id = 'bigquery-public-data.ghcn_d'
tables = client.list_tables(dataset_id)  # Make an API request.

tables_in_noaa_dataset = []
for table in tables:
    tables_in_noaa_dataset.append("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

In [25]:
## We have data from 1763 to 2022
print("\n".join(tables_in_noaa_dataset[0:7]))
print('...')
print("\n".join(tables_in_noaa_dataset[(len(tables_in_noaa_dataset)-7):(len(tables_in_noaa_dataset))]))

bigquery-public-data.ghcn_d.ghcnd_1763
bigquery-public-data.ghcn_d.ghcnd_1764
bigquery-public-data.ghcn_d.ghcnd_1765
bigquery-public-data.ghcn_d.ghcnd_1766
bigquery-public-data.ghcn_d.ghcnd_1767
bigquery-public-data.ghcn_d.ghcnd_1768
bigquery-public-data.ghcn_d.ghcnd_1769
...
bigquery-public-data.ghcn_d.ghcnd_2020
bigquery-public-data.ghcn_d.ghcnd_2021
bigquery-public-data.ghcn_d.ghcnd_2022
bigquery-public-data.ghcn_d.ghcnd_countries
bigquery-public-data.ghcn_d.ghcnd_inventory
bigquery-public-data.ghcn_d.ghcnd_states
bigquery-public-data.ghcn_d.ghcnd_stations


### Sample a table

Include where qflag (quality flag) is null.

In [27]:
_2021_data = """
SELECT
  *
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2021`
WHERE qflag IS NULL
LIMIT 10
"""
_2021_data_query_job = client.query(_2021_data)  # Make an API request.

_2021_data_query_job.to_dataframe()

,id,date,element,value,mflag,qflag,sflag,time,source_url,etl_timestamp
0,USW00063893,2021-02-10,PRCP,0.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
1,USW00054795,2021-02-11,PRCP,0.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
2,USW00023908,2021-02-13,PRCP,0.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
3,USW00094081,2021-02-13,PRCP,13.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
4,USW00004223,2021-02-10,TMIN,-54.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
5,USW00054808,2021-02-10,PRCP,16.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
6,USW00093243,2021-02-10,TMIN,66.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
7,USW00093245,2021-02-12,TMIN,93.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
8,USW00073802,2021-02-13,TMAX,96.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
9,USW00063856,2021-02-10,PRCP,2.0,None,None,R,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00


## Find the stations within lat/long bounds

There are two ways to find this:
Manually establish bounds, i.e. by looking on Google Maps, for example:

burlington: 44.47   73.15 (per https://www.w3.org/2003/01/geo/test/ustowns/latlong.htm)
```
  latitude > 44.3
  AND latitude < 44.7
  AND longitude > -73.4
  AND longitude < -72.9
```
chicago:
```
  latitude > 41.7
  AND latitude < 42
  AND longitude > -87.7
  AND longitude < -87.5
```

Or use the legacy syntax to find those closest to central coordinates as described here: https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/blogs/ghcn/ghcn_on_bq.ipynb

In [28]:
burlington_query = """
SELECT
  id,
  name,
  state,
  latitude,
  longitude
FROM
  `bigquery-public-data.ghcn_d.ghcnd_stations`
WHERE
  latitude > 44.3
  AND latitude < 44.7
  AND longitude > -73.4
  AND longitude < -72.9
"""
burlington_query_job = client.query(burlington_query)  # Make an API request.

burlington_stations = burlington_query_job.to_dataframe()

In [61]:
lat=44.47
lon=-73.15

close_to_query = f"""
SELECT
  name, id,
  state,
  latitude,
  longitude,
  DEGREES(ACOS(SIN(RADIANS(latitude)) * SIN(RADIANS({lat})) + COS(RADIANS(latitude)) * COS(RADIANS({lat})) * COS(RADIANS(longitude - {lon})))) * 60 * 1.515 * 1.609344 AS dist_kms
FROM
  [bigquery-public-data:ghcn_d.ghcnd_stations]
ORDER BY
  dist_kms ASC
LIMIT
  1000"""


In [62]:
# Set use_legacy_sql to True to use legacy SQL syntax (this is necessary to use the DEGREES function)
job_config = bigquery.QueryJobConfig(use_legacy_sql=True)

burlington_query_close_to = client.query(close_to_query, job_config=job_config)  # Make an API request.

burlington_stations_close_to = burlington_query_close_to.to_dataframe()

In [75]:
## list those stations within 30 kms
burlington_stations_close_to[burlington_stations_close_to.dist_kms <= 30]

,name,id,state,latitude,longitude,dist_kms
0,BURLINGTON INTL AP,USW00014742,VT,44.4683,-73.1500,0.248692
1,BURLINGTON,USC00431072,VT,44.4833,-73.1833,3.983434
2,ESSEX JUNCTION,USC00432828,VT,44.4833,-73.1167,3.983434
3,ESSEX JUNCTION 0.1 E,US1VTCH0040,VT,44.4908,-73.1086,5.285001
4,SOUTH BURLINGTON 2.5 SE,US1VTCH0006,VT,44.4277,-73.1502,6.188076
5,ESSEX JUNCTION VERMONT,USR0000VESS,VT,44.5078,-73.1156,6.592889
6,ESSEX JUNCTION 1 N,USC00432843,VT,44.5078,-73.1153,6.609990
7,BURLINGTON 0.5 NW,US1VTCH0005,VT,44.4823,-73.2191,7.433959
8,BURLINGTON 0.4 W,US1VTCH0020,VT,44.4766,-73.2210,7.474223
9,COLCHESTER 1.6 SSW,US1VTCH0031,VT,44.5225,-73.1623,7.786698


## Sample recent weather data from the stations near Burlington, VT

In [77]:
## Build query
stations_within_30_kms = list(burlington_stations_close_to[burlington_stations_close_to.dist_kms <= 30]['id'])

burlington_2022_sample = """
SELECT
  *
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2022`
WHERE qflag IS NULL
AND id IN ('{}')
LIMIT 10
""".format('\', \''.join(stations_within_30_kms))

In [78]:
burlington_2022_sample

"\nSELECT\n  *\nFROM\n  `bigquery-public-data.ghcn_d.ghcnd_2022`\nWHERE qflag IS NULL\nAND id IN ('USW00014742', 'USC00431072', 'USC00432828', 'US1VTCH0040', 'US1VTCH0006', 'USR0000VESS', 'USC00432843', 'US1VTCH0005', 'US1VTCH0020', 'US1VTCH0031', 'US1VTCH0047', 'US1VTCH0027', 'US1VTCH0038', 'US1VTCH0050', 'US1VTCH0019', 'US1VTCH0037', 'US1VTCH0003', 'US1VTCH0013', 'USC00431320', 'US1VTCH0015', 'US1VTCH0012', 'USC00309495', 'USC00434052', 'USC00437607', 'US1VTCH0007', 'USC00309494', 'US1VTCH0004')\nLIMIT 10\n"

In [79]:
burlington_2022_sample = client.query(burlington_2022_sample)  # Make an API request.

burlington_2022_sample_df = burlington_2022_sample.to_dataframe()

In [80]:
burlington_2022_sample_df

,id,date,element,value,mflag,qflag,sflag,time,source_url,etl_timestamp
0,USC00431320,2022-04-27,MDPR,56.0,None,None,7,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:50:47.255695+00:00
1,US1VTCH0003,2022-04-23,MDPR,51.0,None,None,N,0730,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:50:47.255695+00:00
2,US1VTCH0050,2022-07-16,MDPR,0.0,None,None,N,0700,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:53:54.135227+00:00
3,USW00014742,2022-03-17,RHAV,86.0,None,None,W,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:49:22.415865+00:00
4,USW00014742,2022-04-13,ASLP,10142.0,None,None,W,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:50:30.228776+00:00
5,USW00014742,2022-01-26,ASLP,10291.0,None,None,W,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:47:22.781642+00:00
6,USW00014742,2022-01-26,RHAV,75.0,None,None,W,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:47:22.781642+00:00
7,USW00014742,2022-03-14,ADPT,-56.0,None,None,W,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:49:05.353215+00:00
8,USW00014742,2022-03-27,WT01,1.0,None,None,W,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:49:39.519451+00:00
9,USW00014742,2022-03-07,RHMX,58.0,None,None,W,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:49:05.353215+00:00


In [119]:
burlington_2021_time_series = """SELECT
*
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2021`
WHERE qflag IS NULL
AND id IN ('{}')
""".format('\', \''.join(stations_within_30_kms))

burlington_2021_data = client.query(burlington_2021_time_series)

burlington_2021_data_time_series = burlington_2021_data.to_dataframe()

In [126]:
burlington_2021_data_time_series['element'] == 'SNOW'

,id,date,element,value,mflag,qflag,sflag,time,source_url,etl_timestamp
0,USC00431320,2021-02-10,SNOW,0.0,None,None,7,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
1,US1VTCH0015,2021-02-10,SNOW,86.0,None,None,N,0800,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
2,US1VTCH0015,2021-02-11,PRCP,0.0,T,None,N,0800,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
3,US1VTCH0038,2021-02-11,PRCP,0.0,None,None,N,0700,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
4,US1VTCH0015,2021-02-10,PRCP,38.0,None,None,N,0800,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:31:13.158914+00:00
...,...,...,...,...,...,...,...,...,...,...
15393,US1VTCH0003,2021-11-11,PRCP,0.0,None,None,N,0730,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:41:40.405498+00:00
15394,US1VTCH0004,2021-11-11,PRCP,3.0,None,None,N,0700,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:41:40.405498+00:00
15395,USW00014742,2021-11-05,RHMN,74.0,None,None,W,None,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:41:40.405498+00:00
15396,USW00014742,2021-11-06,TMIN,-27.0,None,None,W,2400,ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by...,2022-10-21 03:41:40.405498+00:00


In [130]:
## Visualize
burlington_2021_data_time_series['date'] = pd.to_datetime(burlington_2021_data_time_series['date'])

burlington_snow_2021 = burlington_2021_data_time_series[burlington_2021_data_time_series['element'] == 'SNOW']

fig = px.scatter(burlington_snow_2021,x="date", y="value", color="id", title='Burlington VT Snow')
fig.show()

In [134]:
fig = px.scatter(burlington_snow_2021.groupby(['date']).median().reset_index(),
                 x="date", y="value", title='Burlington VT Median Snow')
fig.show()

# Create a model

https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-glm

https://cloud.google.com/bigquery-ml/docs/arima-multiple-time-series-forecasting-tutorial

Example Time Series (ARIMA):
```
CREATE MODEL `project_id.mydataset.mymodel`
 OPTIONS(MODEL_TYPE='ARIMA_PLUS',
         time_series_timestamp_col='date',
         time_series_data_col='transaction') AS
SELECT
  date,
  transaction
FROM
  `mydataset.mytable`
```

Example linear regression (https://cloud.google.com/bigquery-ml/docs/linear-regression-tutorial):
```
#standardSQL
'''
CREATE OR REPLACE MODEL `bqml_tutorial.penguins_model`
OPTIONS
  (model_type='linear_reg',
  input_label_cols=['body_mass_g']) AS
SELECT
  *
FROM
  `bigquery-public-data.ml_datasets.penguins`
WHERE
  body_mass_g IS NOT NULL
'''
```

#### About  models:
For Time Series (https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series)
* time_series_timestamp_col is what you'd expect
* time_series_data_col is what you're trying to predict

For Linear Regression:

* `input_label_cols` is the thing we're trying to predict
* The items in the `SELECT` statement are what's used to predict the input label

In [139]:
stations_within_30km_text = '\', \''.join(stations_within_30_kms)
# standardSQL
snow_in_burlington_vt_arima = """
CREATE OR REPLACE MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`
 OPTIONS(MODEL_TYPE='ARIMA_PLUS',
         time_series_timestamp_col='date',
         time_series_data_col='mean_value') AS
SELECT
  date,
  AVG(value) as mean_value
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2021`
WHERE qflag IS NULL
AND id IN ('{}')
AND element = 'SNOW'
GROUP BY date
UNION ALL
SELECT
  date,
  AVG(value) as mean_value
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2020`
WHERE qflag IS NULL
AND id IN ('{}')
AND element = 'SNOW'
GROUP BY date
UNION ALL
SELECT
  date,
  AVG(value) as mean_value
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2019`
WHERE qflag IS NULL
AND id IN ('{}')
AND element = 'SNOW'
GROUP BY date
""".format(stations_within_30km_text, stations_within_30km_text,
          stations_within_30km_text)


In [140]:
snow_in_burlington_vt_arima

"\nCREATE OR REPLACE MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`\n OPTIONS(MODEL_TYPE='ARIMA_PLUS',\n         time_series_timestamp_col='date',\n         time_series_data_col='mean_value') AS\nSELECT\n  date,\n  AVG(value) as mean_value\nFROM\n  `bigquery-public-data.ghcn_d.ghcnd_2021`\nWHERE qflag IS NULL\nAND id IN ('USW00014742', 'USC00431072', 'USC00432828', 'US1VTCH0040', 'US1VTCH0006', 'USR0000VESS', 'USC00432843', 'US1VTCH0005', 'US1VTCH0020', 'US1VTCH0031', 'US1VTCH0047', 'US1VTCH0027', 'US1VTCH0038', 'US1VTCH0050', 'US1VTCH0019', 'US1VTCH0037', 'US1VTCH0003', 'US1VTCH0013', 'USC00431320', 'US1VTCH0015', 'US1VTCH0012', 'USC00309495', 'USC00434052', 'USC00437607', 'US1VTCH0007', 'USC00309494', 'US1VTCH0004')\nAND element = 'SNOW'\nGROUP BY date\nUNION ALL\nSELECT\n  date,\n  AVG(value) as mean_value\nFROM\n  `bigquery-public-data.ghcn_d.ghcnd_2020`\nWHERE qflag IS NULL\nAND id IN ('USW00014742', 'USC00431072', 'USC00432828', 'US1VTCH0040', 'US1VTCH0006'

In [141]:
burlington_2019_to_2021_model = client.query(snow_in_burlington_vt_arima)  # Make an API request.

""


## Get the training info

For Arima, it's somewhat limited:
https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-train


In [142]:
burlington_arima_training_info = """
SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`)
"""

In [143]:
burlington_2019_to_2021_model_training = client.query(burlington_arima_training_info)  # Make an API request.
burlington_2019_to_2021_model_training.to_dataframe()

,training_run,iteration,duration_ms
0,0,0,4426


### Get the coefficients

In [145]:
burlington_arima_coefficents_query = """
SELECT
 *
FROM
 ML.ARIMA_COEFFICIENTS(MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`)
 """

In [146]:
burlington_arima_coefficents = client.query(burlington_arima_coefficents_query) 
burlington_arima_coefficents.to_dataframe()

,ar_coefficients,ma_coefficients,intercept_or_drift
0,"[0.7538376047995379, -0.76648692698281]","[-1.630989197493573, 1.369793703112539, -0.687...",0.0


## Evaluate the Model

ARIMA has a special function: https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-arima-evaluate


In [147]:
burlington_arima_eval = """SELECT
  *
FROM
  ML.ARIMA_EVALUATE(MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`, 
  STRUCT(FALSE AS show_all_candidate_models))"""

In [148]:
burlington_2019_to_2021_model_eval = client.query(burlington_arima_eval)  # Make an API request.
burlington_2019_to_2021_model_eval.to_dataframe()

,non_seasonal_p,non_seasonal_d,non_seasonal_q,has_drift,log_likelihood,AIC,variance,seasonal_periods,has_holiday_effect,has_spikes_and_dips,has_step_changes,error_message
0,2,1,3,False,-772.500198,1557.000396,0.239539,[NO_SEASONALITY],False,True,True,


# Build a Forecast

https://cloud.google.com/bigquery-ml/docs/arima-single-time-series-forecasting-tutorial

This is 30 days into the future

```
SELECT
 *
FROM
 ML.FORECAST(MODEL bqml_tutorial.ga_arima_model,
             STRUCT(30 AS horizon, 0.8 AS confidence_level))
```

In [154]:
burlington_arima_forecast_query = """SELECT
 *
FROM
 ML.FORECAST(MODEL `msds-434-robords-oct.weather_prediction.burlington_snow_arima`,
             STRUCT(365 AS horizon, 0.8 AS confidence_level))"""

In [155]:
burlington_arima_forecast_365 = client.query(burlington_arima_forecast_query)  # Make an API request.
burlington_arima_forecast_365_df = burlington_arima_forecast_365.to_dataframe()

In [156]:
burlington_arima_forecast_365_df

,forecast_timestamp,forecast_value,standard_error,confidence_level,prediction_interval_lower_bound,prediction_interval_upper_bound,confidence_interval_lower_bound,confidence_interval_upper_bound
0,2022-01-01 00:00:00+00:00,0.532105,0.489427,0.8,-0.095481,1.159692,-0.095481,1.159692
1,2022-01-02 00:00:00+00:00,0.570822,0.493107,0.8,-0.061483,1.203126,-0.061483,1.203126
2,2022-01-03 00:00:00+00:00,0.639229,0.494130,0.8,0.005614,1.272845,0.005614,1.272845
3,2022-01-04 00:00:00+00:00,0.661122,0.494139,0.8,0.027494,1.294750,0.027494,1.294750
4,2022-01-05 00:00:00+00:00,0.625192,0.494150,0.8,-0.008450,1.258834,-0.008450,1.258834
...,...,...,...,...,...,...,...,...
360,2022-12-27 00:00:00+00:00,0.609070,0.683472,0.8,-0.267337,1.485477,-0.267337,1.485477
361,2022-12-28 00:00:00+00:00,0.609070,0.683926,0.8,-0.267920,1.486060,-0.267920,1.486060
362,2022-12-29 00:00:00+00:00,0.609070,0.684381,0.8,-0.268502,1.486642,-0.268502,1.486642
363,2022-12-30 00:00:00+00:00,0.609070,0.684835,0.8,-0.269084,1.487224,-0.269084,1.487224


In [160]:
burlington_2022_time_series_query = """SELECT
  date,
  AVG(value) as mean_value
FROM
  `bigquery-public-data.ghcn_d.ghcnd_2022`
WHERE qflag IS NULL
AND id IN ('{}')
AND element = 'SNOW'
GROUP BY date
""".format(stations_within_30km_text)

burlington_2022_time_series_data = client.query(burlington_2022_time_series_query)

burlington_2022_time_series_data_df = burlington_2022_time_series_data.to_dataframe()

In [171]:
fig = px.scatter(burlington_2022_time_series_data_df,x="date", y="mean_value", title='Burlington VT Snow 2022')

fig.add_trace(go.Scatter(x=burlington_arima_forecast_365_df["forecast_timestamp"], 
                         y=burlington_arima_forecast_365_df["forecast_value"], 
                         mode="lines", name='Predicted Value'))
fig.show()